In [ ]:
%load_ext autoreload
%autoreload 2
from main import init_model, training_loop

tokenizer, model = init_model()

In [ ]:
from dataset import create_connections_datasets
from main import get_available_device
train_dataset, _ = create_connections_datasets()
device = get_available_device()
training_loop(model, tokenizer, train_dataset, device)